### 1.output parser 


In [1]:
# we use to generate detail report on topic to llm 
# then generate summary of detail report through llm

In [2]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq

load_dotenv()

llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct-0905",  # ✅ spelling fixed
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.3,
)

response = llm.invoke("Hi")
print(response.content)


Hi! How can I help you today?


In [3]:
from langchain_core.prompts import PromptTemplate

In [4]:
t1=PromptTemplate(
    template="write a detailed report on {topic}",
    input_variables=['topic']
)

In [5]:
t2=PromptTemplate(
template="write a 5 line summary on the followrin text reoprt on {topic}",
input_variables=['topic']    
)

In [6]:
p1=t1.invoke({'topic':'black hole'})

In [7]:
result=llm.invoke(p1)

In [8]:
result.content

'**Comprehensive Report on Black Holes**  \n*Prepared by: [Your Name]*  \n*Date: [Insert Date]*  \n\n---\n\n### **1. Executive Summary**  \nBlack holes are regions of spacetime where gravity is so intense that nothing—not even light—can escape. Predicted by Einstein’s general relativity (1915) and confirmed via observations (e.g., LIGO, Event Horizon Telescope), they range from stellar-mass (3–100 M☉) to supermassive (10⁶–10¹⁰ M☉). Key discoveries include Hawking radiation (1974), gravitational waves (2015), and the first direct image of a black hole’s shadow (M87*, 2019). This report synthesizes their formation, properties, observational evidence, and unresolved questions.\n\n---\n\n### **2. Introduction**  \nBlack holes challenge our understanding of physics, acting as laboratories for extreme gravity, quantum mechanics, and thermodynamics. They influence galaxy evolution, star formation, and may hold keys to unifying general relativity with quantum field theory.\n\n---\n\n### **3. F

In [9]:
p2=t2.invoke({"topic":result.content})


In [10]:
re1=llm.invoke(p2)

In [11]:
print(re1.content)

Black holes are ultra-dense spacetime regions where gravity traps even light, forming from stellar collapse, primordial fluctuations, or direct gas collapse.  
General relativity predicts their structures—Schwarzschild (non-spinning) and Kerr (spinning)—while Hawking radiation implies they evaporate.  
Observations confirm them via gravitational waves (LIGO/Virgo), X-ray accretion disks, star motions near Milky Way’s center, and EHT images of M87* and Sgr A*.  
Thermodynamics equates horizon area with entropy, but Hawking’s radiation creates the unresolved information paradox and singularity issues.  
Future missions like LISA and ngEHT will map mergers, test strong-field GR, and probe quantum gravity, potentially linking black holes to dark matter.


### 2.now using chain

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [13]:

parser=StrOutputParser()

In [14]:
chain=t1 | llm | t2 | llm | parser

# here parser is using for extract string 

In [15]:

chain.invoke({"topic":"black hole"})

'Black holes are ultra-dense spacetime regions where gravity traps even light.  \nBorn from stellar collapse, mergers, or primordial fluctuations, they range from 3 M☉ to billions.  \nKey evidence: X-ray disks, 90+ gravitational-wave mergers, and EHT horizon shadows.  \nThey regulate galaxies via feedback and offer labs for quantum gravity and Hawking radiation.  \nNext-gen missions like LISA and ngEHT will test relativity, dark-matter models, and near-horizon physics.'

### 3.json output parser

<!-- it mainly use for give output in json format -->

In [16]:
# it is mainly use for give output in json foramat
print(type(parser))


<class 'langchain_core.output_parsers.string.StrOutputParser'>


In [17]:
from langchain_core.output_parsers import JsonOutputParser

In [18]:
parser2=JsonOutputParser()

In [19]:
t3=PromptTemplate(
    template="give the age,name and city of a fictional person \n {fromat_instruction}",
    input_variables=[],
    partial_variables={"fromat_instruction":parser2.get_format_instructions()}
)

In [20]:
p3=t3.format()

In [21]:
print(p3)

give the age,name and city of a fictional person 
 Return a JSON object.


In [22]:
print(llm.invoke(p3))

content='{"name":"Lila Moreau","age":34,"city":"Montpellier"}' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 42, 'total_tokens': 61, 'completion_time': 0.077379207, 'completion_tokens_details': None, 'prompt_time': 0.038200434, 'prompt_tokens_details': None, 'queue_time': 0.52694492, 'total_time': 0.115579641}, 'model_name': 'moonshotai/kimi-k2-instruct-0905', 'system_fingerprint': 'fp_05df423bab', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019c27dd-c0d8-7bb3-a645-b75d9817ee01-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 42, 'output_tokens': 19, 'total_tokens': 61}


In [23]:
res=llm.invoke(p3)

In [24]:
final_result=parser2.parse(res.content)

In [25]:
print(final_result)

{'name': 'Elena Voss', 'age': 34, 'city': 'Lumenport'}


### *** most important understand promt_templet

In [ ]:
t4=PromptTemplate(
    template="give the age,name and city of a fictional person \n {fromat_instruction}",
    input_variables=["fromat_instruction"]
    # partial_variables={"fromat_instructiongi":parser2.get_format_instructions()}
)

In [32]:
p4=t4.invoke({"fromat_instruction":"return a json object"})

In [33]:
re333=llm.invoke(p4)

In [34]:
print(re333)

content='{"name": "Elara Moonwhisper", "age": 27, "city": "Lunaris"}' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 41, 'total_tokens': 66, 'completion_time': 0.08932636, 'completion_tokens_details': None, 'prompt_time': 0.010892912, 'prompt_tokens_details': None, 'queue_time': 0.282576465, 'total_time': 0.100219272}, 'model_name': 'moonshotai/kimi-k2-instruct-0905', 'system_fingerprint': 'fp_05df423bab', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019c27ec-c821-75f1-a616-e429279c26c8-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 41, 'output_tokens': 25, 'total_tokens': 66}


In [35]:
final_result333=parser2.parse(re333.content)

In [36]:
print(final_result333)

{'name': 'Elara Moonwhisper', 'age': 27, 'city': 'Lunaris'}


### now using chain


In [47]:
chain2=t3 | llm | parser2

In [48]:
print(chain2)

first=PromptTemplate(input_variables=[], input_types={}, partial_variables={'fromat_instruction': 'Return a JSON object.'}, template='give the age,name and city of a fictional person \n {fromat_instruction}') middle=[ChatGroq(profile={'max_input_tokens': 262144, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001C764EAC4D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C765009040>, model_name='moonshotai/kimi-k2-instruct-0905', temperature=0.3, model_kwargs={}, groq_api_key=SecretStr('**********'))] last=JsonOutputParser()


In [50]:
reee=chain2.invoke({})